### `INTRODUCTION`
`The theory of natural evolution`, proposed by `Charles Darwin`, states that the fittest individuals in a population are selected for offspring reproduction. This process repeats until the new generation is able to fit the environment. When the environment changes and challenges the finest generation, the individuals keep evolving.\

Inspired by `The theory of natural evolution`, `Genetic Algorithm` helps find the best solution that can fit all requirements. There are several topics that apply `Genetic Algorithm`, such as:
- `Travelling Salesman Problem`: find the shortest way to move between multiple locations
- `Optimization problems`
- `Evolutionary robotics`
- `Scheduling`
- ...

In this analysis, the dummy data I used describes a store with 10 employees (7 staff, 3 leaders & managers) who can work in 2 work shifts (morning, evening). The output is I use `Genetic Algorithm` to make a timetable for scheduling the work shift.\

To finish to analysis, I definitely used multiple references. For the theory of `Genetic Algorithm`, I referred to:
- https://towardsdatascience.com/introduction-to-genetic-algorithms-including-example-code-e396e98d8bf3
- https://github.com/hassanzadehmahdi/Traveling-Salesman-Problem-using-Genetic-Algorithm/blob/main/tsp.py

For the Python code, I inspired (but do not use them all because they cannot totally apply in my case) by:
- https://www.youtube.com/watch?v=8NrNX_jCkjw

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from data import SHIFTS
from display import DisplayMgr
from population import (
    Population, 
    Schedule, 
    POPULATION_SIZE
)
from algorithm import (
    GeneticAlgorithm, 
    NUMB_OF_ELITE_SCHEDULES,
    TOURNAMENT_SELECTION_SIZE,
    MUTATION_RATE
)

### `Data`
- `Employees`: 10 employees
    - `employeeCode`: each employee has their own unique id
    - `jobTitleName`: there are 1 manager, 2 leaders, and 7 staff
    
- `Shifts`: 60 shifts
    - `shiftName`: From 30 days in a month, there are 2 work shifts in each day: MORNING (MOR) and EVENING (EVE)
    - `minEmployees`: This is the minimum number of employees in each work shift
    - `maxEmployees`: This is the maximum number of employees in each work shift

In [2]:
# initialize the `DisplayMgr` object
displayMgr = DisplayMgr()

In [3]:
# print all employees
displayMgr.print_employees()


> Employees
+--------------+--------------+
| employeeCode | jobTitleName |
+--------------+--------------+
|      E0      |   Manager    |
|      E1      |    Leader    |
|      E2      |    Leader    |
|      E3      |    Staff     |
|      E4      |    Staff     |
|      E5      |    Staff     |
|      E6      |    Staff     |
|      E7      |    Staff     |
|      E8      |    Staff     |
|      E9      |    Staff     |
+--------------+--------------+


In [4]:
# print top 10 work shifts
displayMgr.print_shifts(10)


> Shifts Info
+-----------+--------------+--------------+
| shiftName | minEmployees | maxEmployees |
+-----------+--------------+--------------+
|   01-MOR  |      3       |      5       |
|   01-EVE  |      4       |      6       |
|   02-MOR  |      4       |      6       |
|   02-EVE  |      5       |      7       |
|   03-MOR  |      4       |      6       |
|   03-EVE  |      5       |      7       |
|   04-MOR  |      3       |      5       |
|   04-EVE  |      4       |      6       |
|   05-MOR  |      3       |      5       |
|   05-EVE  |      4       |      6       |
+-----------+--------------+--------------+


### `Step 1`: INITIAL POPULATION
From data we have, we will randomly generate:
- a `population` which is a list of,
- `individuals` representing scheduling solutions, which contains,
- `chromosomes` representing work shifts with assigned employees

In [5]:
print('The default number of individuals in a population is:', POPULATION_SIZE)
population = Population()

The default number of individuals in a population is: 10


### `Step 2`: FITNESS FUNCTION
This step evaluates the fitness ability of each individual.\
I assume to have 3 conditions that each individual needs to fit:
- There is at least 1 manager or leader in each shift (chromosome)
- 1 employee must work from 23 shifts to 30 shifts in a month
- The number of employees is in control in each shift

The conflicts contain any unadaptable condition cases. The fitness is calculated as 1 / (1 + conflicts), and expected to be 1 when there are no more conflicts.

In [6]:
# as we can see here, the population has 10 individuals (schedules)
# each individual has their own number of conflicts (with conditions)
# for example, schedule 0 has fitness equal to 0.048, 20 conflicts:
## 05 from condition 1 (There are 8 shifts without manager or leader), 
## 05 from condition 2 (There are 3 employees not working from 23 to 30 shifts in a month)
## 10 from condition 3 (There are 20 shifts with less or more employees as accepted)

displayMgr.print_generation(population)

+------------+---------+----------------+-------------------+
| schedule # | fitness | # of conflicts | conflicts details |
+------------+---------+----------------+-------------------+
|     0      |  0.048  |       20       |     [5, 5, 10]    |
|     1      |  0.036  |       27       |    [11, 4, 12]    |
|     2      |  0.036  |       27       |    [11, 1, 15]    |
|     3      |  0.036  |       27       |     [9, 3, 15]    |
|     4      |  0.036  |       27       |    [11, 4, 12]    |
|     5      |  0.036  |       27       |    [11, 4, 12]    |
|     6      |  0.034  |       28       |    [11, 3, 14]    |
|     7      |  0.033  |       29       |    [12, 4, 13]    |
|     8      |  0.031  |       31       |    [10, 4, 17]    |
|     9      |   0.03  |       32       |    [16, 3, 13]    |
+------------+---------+----------------+-------------------+


### `Step 3`: SELECTION
This step select the 2 fittest parent individuals for the crossover (next step).

In [7]:
# initialize the `GeneticAlgorithm` object
geneticAlgorithm = GeneticAlgorithm()

In [8]:
# randomly select some individuals from population
print("The number of individuals randomly selected from the population:", TOURNAMENT_SELECTION_SIZE)
tournament_population = geneticAlgorithm.select_tournament_population(population)
displayMgr.print_generation(tournament_population)

# then, the fittest individual is selected
schedule1 = tournament_population.get_schedules()[0]
assert isinstance(schedule1, Schedule)
print(schedule1)
print('finess:', schedule1.get_fitness())
print('# of conflicts:', schedule1.get_numbOfConflicts())
print('conflicts details:', schedule1.get_conflicts())

The number of individuals randomly selected from the population: 4
+------------+---------+----------------+-------------------+
| schedule # | fitness | # of conflicts | conflicts details |
+------------+---------+----------------+-------------------+
|     0      |  0.034  |       28       |    [11, 3, 14]    |
|     1      |  0.033  |       29       |    [12, 4, 13]    |
|     2      |  0.031  |       31       |    [10, 4, 17]    |
|     3      |   0.03  |       32       |    [16, 3, 13]    |
+------------+---------+----------------+-------------------+
finess: 0.034482758620689655
# of conflicts: 28
conflicts details: [11, 3, 14]


In [9]:
# repeat the selection and assign to the second parent individual
print("The number of individuals randomly selected from the population:", TOURNAMENT_SELECTION_SIZE)
tournament_population = geneticAlgorithm.select_tournament_population(population)
displayMgr.print_generation(tournament_population)

schedule2 = tournament_population.get_schedules()[0]
assert isinstance(schedule2, Schedule)
print(schedule2)
print('finess:', schedule2.get_fitness())
print('# of conflicts:', schedule2.get_numbOfConflicts())
print('conflicts details:', schedule2.get_conflicts())

The number of individuals randomly selected from the population: 4
+------------+---------+----------------+-------------------+
| schedule # | fitness | # of conflicts | conflicts details |
+------------+---------+----------------+-------------------+
|     0      |  0.048  |       20       |     [5, 5, 10]    |
|     1      |  0.036  |       27       |     [9, 3, 15]    |
|     2      |  0.033  |       29       |    [12, 4, 13]    |
|     3      |  0.031  |       31       |    [10, 4, 17]    |
+------------+---------+----------------+-------------------+
finess: 0.047619047619047616
# of conflicts: 20
conflicts details: [5, 5, 10]


### `Step 4`: CROSSOVER
This step carries on the 2 fittest parent individuals to generate the new offspring.

To form a new generation, the population first select some elite individuals from the last generation, then process the crossover to make remaining offsprings.

In [10]:
# now we can see, there is a new offspring that may be better than its parents
offspring_schedule = geneticAlgorithm.crossover_schedule(schedule1, schedule2)
print(offspring_schedule)
print('finess:', offspring_schedule.get_fitness())
print('# of conflicts:', offspring_schedule.get_numbOfConflicts())
print('conflicts details:', offspring_schedule.get_conflicts())


finess: 0.05263157894736842
# of conflicts: 18
conflicts details: [8, 2, 8]


In [11]:
# the new generation is formed form some elite individuals and new crossover offsprings
print("Number of elite individuals first selected:", NUMB_OF_ELITE_SCHEDULES)
new_generation = geneticAlgorithm.crossover_population(population)
displayMgr.print_generation(new_generation)

Number of elite individuals first selected: 4
+------------+---------+----------------+-------------------+
| schedule # | fitness | # of conflicts | conflicts details |
+------------+---------+----------------+-------------------+
|     0      |  0.048  |       20       |     [5, 5, 10]    |
|     1      |  0.042  |       23       |     [8, 3, 12]    |
|     2      |  0.042  |       23       |     [7, 3, 13]    |
|     3      |  0.036  |       27       |    [11, 4, 12]    |
|     4      |  0.036  |       27       |    [11, 1, 15]    |
|     5      |  0.036  |       27       |     [9, 3, 15]    |
|     6      |  0.036  |       27       |    [11, 1, 15]    |
|     7      |  0.033  |       29       |    [12, 2, 15]    |
|     8      |  0.032  |       30       |    [17, 1, 12]    |
|     9      |  0.028  |       35       |    [15, 5, 15]    |
+------------+---------+----------------+-------------------+


### `Step 5`: MUTATION
After crossover, the mutation possibly appears to make the new generation unlike normal. It is truly necessary for new species creation.

In [12]:
print("The mutation rate:", MUTATION_RATE)

The mutation rate: 0.1


In [13]:
# individual 1 is randomly mutated
mutate_schedule1 = geneticAlgorithm.mutate_schedule(schedule1)
print(mutate_schedule1)
print('finess:', mutate_schedule1.get_fitness())
print('# of conflicts:', mutate_schedule1.get_numbOfConflicts())
print('conflicts details:', mutate_schedule1.get_conflicts())

finess: 0.041666666666666664
# of conflicts: 23
conflicts details: [9, 1, 13]


In [14]:
# the new generation after mutation
mutated_generation = geneticAlgorithm.mutate_population(new_generation)
displayMgr.print_generation(mutated_generation)

+------------+---------+----------------+-------------------+
| schedule # | fitness | # of conflicts | conflicts details |
+------------+---------+----------------+-------------------+
|     0      |  0.048  |       20       |     [5, 5, 10]    |
|     1      |  0.042  |       23       |     [8, 3, 12]    |
|     2      |  0.042  |       23       |     [7, 3, 13]    |
|     3      |   0.04  |       24       |     [8, 3, 13]    |
|     4      |  0.036  |       27       |    [11, 4, 12]    |
|     5      |  0.036  |       27       |     [9, 3, 15]    |
|     6      |  0.033  |       29       |    [11, 2, 16]    |
|     7      |  0.033  |       29       |    [12, 2, 15]    |
|     8      |  0.029  |       33       |    [17, 3, 13]    |
|     9      |  0.026  |       37       |    [15, 6, 16]    |
+------------+---------+----------------+-------------------+


### `FINAL`
Gather all steps and run the whole algorithm to find the best work shift scheduling.

In [15]:
# the i-th of generation
generationNumber = 0
print('\n> Generation # ' + str(generationNumber))

# initial population
population = Population()
displayMgr.print_generation(population)

# get the fittest individual from the initialized population
best_schedule = population.get_schedules()[0]

# check whether `best_schedule` is `Schedule` object
assert isinstance(best_schedule, Schedule)

# run a while loop for `Genetic Algorithm`
while (
    best_schedule.get_fitness() != 1.0 and # when we find the fittest schedule
    generationNumber < 500 # the process should be stop at 500-th generation because the conditions are too hard to be fitted
):
    # mark the next generation
    generationNumber += 1

    # evolve the population, including steps 3,4,5
    population = geneticAlgorithm.evolve(population)

    # reassign the fittest individual from the new generation
    best_schedule = population.get_schedules()[0]

    # print out the new generation
    print("\n> Generation # " + str(generationNumber))
    displayMgr.print_generation(population)
else:
    # if the while loop stops, save the fittest individual
    df_chedule = displayMgr.print_schedule_as_table(best_schedule)
print('\n\n')


> Generation # 0
+------------+---------+----------------+-------------------+
| schedule # | fitness | # of conflicts | conflicts details |
+------------+---------+----------------+-------------------+
|     0      |  0.048  |       20       |     [7, 5, 8]     |
|     1      |   0.04  |       24       |    [10, 2, 12]    |
|     2      |  0.034  |       28       |    [13, 5, 10]    |
|     3      |  0.034  |       28       |     [6, 5, 17]    |
|     4      |  0.032  |       30       |    [11, 7, 12]    |
|     5      |  0.031  |       31       |     [8, 6, 17]    |
|     6      |   0.03  |       32       |    [12, 5, 15]    |
|     7      |   0.03  |       32       |    [16, 3, 13]    |
|     8      |   0.03  |       32       |    [15, 2, 15]    |
|     9      |  0.028  |       35       |    [12, 5, 18]    |
+------------+---------+----------------+-------------------+

> Generation # 1
+------------+---------+----------------+-------------------+
| schedule # | fitness | # of conf

In [16]:
# now, check whethe the evolution stops, and we get the fittest individual
if generationNumber < 500 and best_schedule.get_fitness() == 1.0:
    print("The fittest individual has been found is in the generation of", generationNumber)
else:
    print("The evolution should have continued...but we still get one!")

# print out the fittest schedule
df_chedule.head(10)

The fittest individual has been found is in the generation of 152


,shiftName,EmployeeCode,JobTitleName
0,01-MOR,E1,Leader
1,01-MOR,E8,Staff
2,01-MOR,E6,Staff
3,01-MOR,E3,Staff
4,01-EVE,E9,Staff
5,01-EVE,E0,Manager
6,01-EVE,E1,Leader
7,01-EVE,E4,Staff
8,01-EVE,E5,Staff
9,02-MOR,E1,Leader


In [17]:
# recheck the condition 1
# there is always at least manager or leader in each work shift
df_chedule[df_chedule['JobTitleName'].isin(['Manager','Leader'])].nunique()

shiftName       60
EmployeeCode     3
JobTitleName     2
dtype: int64

In [18]:
# recheck the condition 2
# each employee has at least 24 work shifts and maximum 30 work shifts
(
    df_chedule
    .groupby('EmployeeCode', as_index=False)
    .agg(numbShifts=('shiftName','nunique'))
)

,EmployeeCode,numbShifts
0,E0,30
1,E1,29
2,E2,25
3,E3,26
4,E4,24
5,E5,29
6,E6,26
7,E7,24
8,E8,26
9,E9,27


In [19]:
# recheck the condition 3
# there are enough employees in each work shift
(
    df_chedule
    .groupby("shiftName", as_index=False)
    .agg(employees=("EmployeeCode","nunique"))
    .merge(pd.DataFrame(SHIFTS, columns=["shiftName","minEmployees","maxEmployees"]))
    .query("~employees.between(minEmployees, maxEmployees)")
    .shape
)

(0, 4)

### CONCLUSION
Finally, we scheduled a timetable for the employee's work shift. \
Please note that the data are all dummies, and the given conditions can be stricter or easier, up to the business analysis context.